In [1]:
import torch
import dataset

In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('using device:', device)

using device: cpu


In [3]:
path = "/home/host/Documents/JupyterProjects/Test_ML_DBA/dataset_test_task/"

loaded_data = dataset.DataLoad(path)
loaded_data.data.head()

,filename,label
0,30900_5_010.jpg,010
1,267155_9_010.jpg,010
2,125867_3_000.jpg,000
3,10762_10_010.jpg,010
4,68924_5_000.jpg,000


In [4]:
data = loaded_data.data
data.shape

(6000, 2)

In [19]:
# Хера себе дисбаланс

# [000] = 2186
# [001] = 545
# [010] = 2008
# [011] = 5
# [100] = 1226
# [101] = 8
# [110] = 22
# [111] = 0 - нет данных

# итого 7 классов
count = 0
for label in data['label']:
    if label == '100':
        count += 1
count

1226

In [ ]:
# Как-то нужно размножить классы [011], [101], [110]


In [20]:
x_train, x_valid, x_test = loaded_data.split_data()

print('Размер тренировочного набора данных', x_train.shape)
print('Размер валидационного набора данных', x_valid.shape)
print('Размер тестового набора данных', x_test.shape)

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

In [ ]:
x_train = x_train.reset_index(drop=True)
x_valid = x_valid.reset_index(drop=True)
x_test = x_test.reset_index(drop=True)

In [ ]:
from dataset import CustomDataset

train_dataset = CustomDataset(data_frame=x_train, img_dir=path, transform=dataset.train_augmentation)
valid_dataset = CustomDataset(data_frame=x_valid, img_dir=path, transform=dataset.test_augmentation)
test_dataset = CustomDataset(data_frame=x_test, img_dir=path, transform=dataset.test_augmentation)

In [ ]:
from torch.utils.data import DataLoader

batch_size = 64 # размер батча
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
import model
from train import train_model

net = model.Model().to(device)

loss_history, best_model, y_pred_valid = train_model(train_loader, valid_loader, net, epochs=8, device=device)

In [ ]:
from train import test_model

y_pred_test = test_model(model=best_model, loader_test=test_loader, device=device)